In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [2]:
from mlxtend.preprocessing import minmax_scaling 

In [3]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [5]:
scaled_data = minmax_scaling(df,columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'])

In [6]:
def build_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(scaled_data.keys())]),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.01)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = build_model()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-23 14:35:49.159764: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-23 14:35:49.160432: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [8]:
EPOCHS = 1000

history = model.fit(scaled_data, df['Outcome'],epochs=EPOCHS, validation_split=0.2, verbose=2)

Epoch 1/1000


2023-03-23 14:36:07.999937: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-23 14:36:08.217313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 - 2s - loss: 0.6595 - accuracy: 0.6531 - val_loss: 0.6456 - val_accuracy: 0.6429 - 2s/epoch - 87ms/step
Epoch 2/1000


2023-03-23 14:36:09.666202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 - 0s - loss: 0.6238 - accuracy: 0.6547 - val_loss: 0.6131 - val_accuracy: 0.6429 - 168ms/epoch - 8ms/step
Epoch 3/1000
20/20 - 0s - loss: 0.5843 - accuracy: 0.6743 - val_loss: 0.5701 - val_accuracy: 0.6883 - 156ms/epoch - 8ms/step
Epoch 4/1000
20/20 - 0s - loss: 0.5385 - accuracy: 0.7248 - val_loss: 0.5502 - val_accuracy: 0.8052 - 162ms/epoch - 8ms/step
Epoch 5/1000
20/20 - 0s - loss: 0.5022 - accuracy: 0.7720 - val_loss: 0.5026 - val_accuracy: 0.7857 - 155ms/epoch - 8ms/step
Epoch 6/1000
20/20 - 0s - loss: 0.4762 - accuracy: 0.7850 - val_loss: 0.4784 - val_accuracy: 0.7857 - 157ms/epoch - 8ms/step
Epoch 7/1000
20/20 - 0s - loss: 0.4569 - accuracy: 0.7980 - val_loss: 0.4620 - val_accuracy: 0.7922 - 153ms/epoch - 8ms/step
Epoch 8/1000
20/20 - 0s - loss: 0.4353 - accuracy: 0.7948 - val_loss: 0.4848 - val_accuracy: 0.7403 - 164ms/epoch - 8ms/step
Epoch 9/1000
20/20 - 0s - loss: 0.4250 - accuracy: 0.7899 - val_loss: 0.4438 - val_accuracy: 0.7987 - 154ms/epoch - 8ms/step
Epoch 10/1000

In [9]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
995,0.267066,0.894137,0.359476,0.850649,995
996,0.269858,0.887622,0.353697,0.857143,996
997,0.269618,0.889251,0.360203,0.863636,997
998,0.269840,0.885994,0.369828,0.850649,998
999,0.267383,0.881108,0.410436,0.850649,999


In [10]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy = {}%".format(round(acc, 2), round(val_acc, 2)))

Training Accuracy = 88.76% and Validation Accuracy = 85.45%
